In [1]:
import pandas as pd

df = pd.read_csv('Resources/charity_data.csv')
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 1: Preprocessing

Target variable: IS_SUCCESSFUL

Features:  
-APPLICATION_TYPE  
-AFFILIATION  
-CLASSIFICATION  
-USE_CASE  
-ORGANIZATION  
-STATUS  
-INCOME_AMT  
-SPECIAL_CONSIDERATIONS  
-ASK_AMT

In [2]:
# Remove columns that won't contribute toward machine learnings model
df.drop(columns=['EIN','NAME'],inplace=True)

In [3]:
# Check number of unique values per variable and add variable names with >10 unique values to list
flag_variables = []

for variable in df.columns:
    print(f'{variable}: {len(df[variable].unique())}')
    if len(df[variable].unique()) > 10:
        flag_variables.append(variable)

APPLICATION_TYPE: 17
AFFILIATION: 6
CLASSIFICATION: 71
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
SPECIAL_CONSIDERATIONS: 2
ASK_AMT: 8747
IS_SUCCESSFUL: 2


In [4]:
# Print value counts for flagged variables
for variable in flag_variables:
    print(df[variable].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [5]:
# Generate list of application types with less than 600 occurrences
application_type = df['APPLICATION_TYPE'].value_counts()
application_other = application_type[application_type < 600].index

In [6]:
# Generate list of classifications with less than 1000 occurrences
classification_type = df['CLASSIFICATION'].value_counts()
classification_other = classification_type[classification_type < 1000].index

In [7]:
# Reassign any application types with less than 600 occurrences to "Other"
df.loc[df['APPLICATION_TYPE'].isin(application_other), 'APPLICATION_TYPE'] = 'Other'
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Reassign any classifications with less than 1000 occurrences to "Other"
df.loc[df['CLASSIFICATION'].isin(classification_other), 'CLASSIFICATION'] = 'Other'
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert 'Y' and 'N' in 'SPECIAL_CONSIDERATIONS' to 1 and 0 respectively
df['SPECIAL_CONSIDERATIONS'] = df['SPECIAL_CONSIDERATIONS']\
                                .apply(lambda value: 1 if value == 'Y' else 0)
df['SPECIAL_CONSIDERATIONS'].value_counts()

0    34272
1       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [13]:
# Use get_dummies to convert categorical variables into dummies
df_clean = pd.get_dummies(df)
df_clean

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,0,5000,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,5000,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [22]:
# Split dataframe between features and target
X = df_clean.drop(columns='IS_SUCCESSFUL').values
y = df_clean['IS_SUCCESSFUL'].values

# Split the data into training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=9)

In [24]:
# Standardize data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Step 2: Compile, Train, and Evaluate Model

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
# Check shape of training data
X_train_scaled.shape

(25724, 41)

In [33]:
# Initialize neural network model and add layers
model = keras.Sequential()

model.add(layers.Dense(6, activation="relu",input_dim = X_train_scaled.shape[1]))
model.add(layers.Dense(6, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

SyntaxError: invalid syntax (2199788812.py, line 1)

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 252       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile, fit, and evaluate model on test data
# mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
#                                      save_weights_only=True, period=5)
# model.fit(X_train, Y_train, callbacks=[mc])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train_scaled, y_train, epochs=100)

model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 0s 407us/step - loss: 0.5559 - accuracy: 0.7251
Epoch 2/100
804/804 [==============================] - 0s 398us/step - loss: 0.5545 - accuracy: 0.7260
Epoch 3/100
804/804 [==============================] - 0s 400us/step - loss: 0.5546 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 0s 402us/step - loss: 0.5549 - accuracy: 0.7256
Epoch 5/100
804/804 [==============================] - 0s 400us/step - loss: 0.5428 - accuracy: 0.7350
Epoch 6/100
804/804 [==============================] - 0s 396us/step - loss: 0.5499 - accuracy: 0.7298
Epoch 7/100
804/804 [==============================] - 0s 399us/step - loss: 0.5482 - accuracy: 0.7310
Epoch 8/100
804/804 [==============================] - 0s 399us/step - loss: 0.5481 - accuracy: 0.7340
Epoch 9/100
804/804 [==============================] - 0s 399us/step - loss: 0.5475 - accuracy: 0.7305
Epoch 10/100
804/804 [==============================] - 0s 398us/step - l

804/804 [==============================] - 0s 398us/step - loss: 0.5457 - accuracy: 0.7338
Epoch 79/100
804/804 [==============================] - 0s 395us/step - loss: 0.5378 - accuracy: 0.7427
Epoch 80/100
804/804 [==============================] - 0s 395us/step - loss: 0.5443 - accuracy: 0.7352
Epoch 81/100
804/804 [==============================] - 0s 396us/step - loss: 0.5499 - accuracy: 0.7307
Epoch 82/100
804/804 [==============================] - 0s 397us/step - loss: 0.5476 - accuracy: 0.7315
Epoch 83/100
804/804 [==============================] - 0s 394us/step - loss: 0.5431 - accuracy: 0.7332
Epoch 84/100
804/804 [==============================] - 0s 395us/step - loss: 0.5462 - accuracy: 0.7328
Epoch 85/100
804/804 [==============================] - 0s 396us/step - loss: 0.5498 - accuracy: 0.72660s - loss: 0.5499 - accuracy: 0.72
Epoch 86/100
804/804 [==============================] - 0s 396us/step - loss: 0.5481 - accuracy: 0.7293
Epoch 87/100
804/804 [=====================